# Approche RFPC 

---

**Requirement**
> - statsmodel
> - fanalysis 
> - pandas
> - numpy

Pour installer directement depuis le notebook : 

```
%pip install fanalysis
```


**Importation**

In [1]:
import pandas as pd
import numpy as np
from S_RFPC import RFPC, verification_bloc

In [2]:
data = pd.read_table("mobil_init.txt").iloc[:, 1:]

In [3]:
data.head()

,CUEX1,CUEX2,CUEX3,PERQ1,PERQ2,PERQ3,PERQ4,PERQ5,PERQ6,PERQ7,PERV1,PERV2,CUSA1,CUSA2,CUSA3,CUSL1,CUSL2,CUSL3
0,6,6,5,6,5,2,6,5,3,3,0,1,5,2,6,5,3,5
1,9,9,8,9,8,9,9,8,9,9,9,9,9,9,7,9,0,9
2,6,6,6,6,7,3,6,7,6,6,6,6,7,6,6,5,0,6
3,6,9,3,7,9,9,7,2,3,7,3,3,9,9,9,9,2,9
4,7,6,9,9,8,7,9,8,8,7,5,5,9,7,7,9,1,7


**Définition des liens**

Il faut pour chaque variable créer un vecteur contenant des 0 ou des 1 afin de définir quelles variables pointent sur quelles autres variables. La valeur 1 signifie que la variable est pointée par une autre. La position du 1 permet de savoir quelle est la variable en question.

_Par exemple, **pqual** est pointée par la variable **expect**, car on trouve un 1 dans la première position du vecteur **pqual**._

In [4]:
expect = [0, 0, 0, 0, 0]
pqual = [1, 0, 0, 0, 0]
pval = [1, 1, 0, 0, 0]
satis = [1, 1, 1, 0, 0]
loyal = [0, 0, 0, 1, 0]

path = np.array([expect, pqual, pval, satis, loyal])
vl = ["expect", "pqual", "pval", "satis", "loyal"]
path = pd.DataFrame(path, columns=vl, index=vl)

On affiche la matrice sous la forme d'un DataFrame (nécéssaire pour la fonction RFPC)

In [5]:
path

,expect,pqual,pval,satis,loyal
expect,0,0,0,0,0
pqual,1,0,0,0,0
pval,1,1,0,0,0
satis,1,1,1,0,0
loyal,0,0,0,1,0


**Définition des blocs**

La seconde étape consiste à l'attribution de chaque variable manifeste à son bloc. Pour cela, les données doivent être de la forme d'un DataFrame.

On crée donc une liste de listes avec les indices de début et de fin des variable de chaque bloc. La fonction _vérification\_bloc_ permet de vérifier si les blocs sont exacts.

In [6]:
block = [[0, 3], [3, 10], [10, 12], [12, 15], [15, 18]]

In [7]:
verification_bloc(block, vl, data)

Bloc : expect 
Variables manifestes : ['CUEX1', 'CUEX2', 'CUEX3'] 

Bloc : pqual 
Variables manifestes : ['PERQ1', 'PERQ2', 'PERQ3', 'PERQ4', 'PERQ5', 'PERQ6', 'PERQ7'] 

Bloc : pval 
Variables manifestes : ['PERV1', 'PERV2'] 

Bloc : satis 
Variables manifestes : ['CUSA1', 'CUSA2', 'CUSA3'] 

Bloc : loyal 
Variables manifestes : ['CUSL1', 'CUSL2', 'CUSL3'] 



__Estimation du modèle__

On renseigne les informations précédente dans la fonction _RFPC_ : 
- **les données** – (Dataframe)
- **blocs** – (liste de liste des indices)
- **nom des variables sous forme de liste** – (liste de _str_ des noms des variables) 
- **matrice des liens** – (matrice des liens sous forme de Dataframe).

In [10]:
model = RFPC(data, block, vl, path, verbose=True)

['CUEX1', 'CUEX2', 'CUEX3']
['PERQ1', 'PERQ2', 'PERQ3', 'PERQ4', 'PERQ5', 'PERQ6', 'PERQ7']
['PERV1', 'PERV2']
['CUSA1', 'CUSA2', 'CUSA3']
['CUSL1', 'CUSL2', 'CUSL3']


On peut récupérer les différentes sorties : modèle interne, modèle externe, récapitulatif du modèle externe et le GOF

In [11]:
model.final_inner

,R2,AVE,Mean_redundancy
expect,0.000,0.475,0.000
pqual,0.284,0.576,0.163
pval,0.334,0.846,0.282
satis,0.653,0.688,0.449
loyal,0.403,0.523,0.211


In [12]:
model.final_outer

,Bloc,weight,loading,communality,redundancy
CUEX1,expect,0.560205,0.798378,0.637407,0.000000
CUEX2,expect,0.504299,0.718702,0.516533,0.000000
CUEX3,expect,0.365421,0.520781,0.271212,0.000000
PERQ1,pqual,0.198873,0.802172,0.643479,0.182574
PERQ2,pqual,0.158860,0.640776,0.410594,0.116498
PERQ3,pqual,0.191658,0.773069,0.597635,0.169567
PERQ4,pqual,0.194011,0.782559,0.612398,0.173756
PERQ5,pqual,0.186375,0.751759,0.565142,0.160348
PERQ6,pqual,0.197203,0.795438,0.632721,0.179522
PERQ7,pqual,0.187440,0.756055,0.571619,0.162185


In [13]:
model.GOF

0.5100388220518121

In [14]:
for i in model.list_resul_inner:
    print(i)
    print('-'*55)

            coef        se      tval          pval
expect  0.896124  0.090231  9.931481  8.506244e-20
-------------------------------------------------------
            coef        se      tval          pval
expect  0.078761  0.066719  1.180499  2.389329e-01
pqual   0.347012  0.039658  8.750106  3.335519e-16
-------------------------------------------------------
            coef        se       tval          pval
expect  0.124906  0.053426   2.337936  2.018955e-02
pqual   0.421849  0.036228  11.644296  2.869509e-25
pval    0.257395  0.050706   5.076189  7.582194e-07
-------------------------------------------------------
          coef        se       tval          pval
satis  0.55333  0.042697  12.959567  1.058381e-29
-------------------------------------------------------
